At its core, LangChain is a framework built around LLMs. 

We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more.


In [ ]:
* At its core, LangChain is a framework built around LLMs. 

* We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more.

* The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs.

* Chains may consist of multiple components from several modules:

* Prompt templates: Prompt templates are templates for different types of prompts. Like “chatbot” style templates, ELI5 question-answering, etc

* LLMs: Large language models like ChatGPT, Bard, Claude, etc.

* Agents: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all are packaged into a logical loop of operations.

* Memory: Short-term memory, long-term memory.

* Here we will only introduce the functionlity that will allow us to turn unstrctued to structured text to get insigh form it. 



In [ ]:
Basic usage is simlar to what we've done in the previous session.

1. Build a prompt
2. Feed the prompt an LLM
3. Get the results back



In [ ]:
prompt = """ What is the most populated city in the state of Hawaii. 
Provide city name and no additional information."""

import os
import openai

# openai.api_key = "ADD API KEY HERE"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": prompt
    }
  ],
  temperature=0,
  max_tokens=128,
)
print(response)


In [ ]:
response["choices"][0]["message"]["content"]

In [ ]:
from langchain.prompts import PromptTemplate

from langchain.chat_models import ChatOpenAI


In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
prompt_str = """What is the most populated city in the state of Hawaii. 
Provide city name and no additional information."""

prompt = PromptTemplate.from_template(prompt_str)


In [ ]:
chain = prompt | model


In [ ]:
chain.invoke({})

In [ ]:
### Prompts Are First Class objects in LangChain

* Prompt can be easily customized to take run time variable.
* Can be customized with exmaples


In [ ]:
prompt_str = """What is the most populated city in the state of {state}.

Provide city name and no additional information."""

prompt = PromptTemplate.from_template(prompt_str)

In [ ]:
chain = prompt | model

In [ ]:
response = chain.invoke({"state": "Hawaii"})
response.content

In [ ]:
response = chain.invoke({"state": "California"})
response.content

In [ ]:
response = chain.invoke({"state": "Georgia"})
response.content

In [ ]:
prompt_str = """What is the most populated city in the state provided below.

Provide city name and no additional information. 

Examples:

State: Hawaii
City: Honolulu

State: California
City: Los Angeles

State: {state}
"""

prompt = PromptTemplate.from_template(prompt_str)
chain = prompt | model


In [ ]:
response = chain.invoke({"state": "Georgia"})
response

In [ ]:
response.content

In [ ]:
prompt_str = """What is the most populated city in the state provided below.

Provide city name and no additional information. 

Examples:

State: Hawaii
{{"City": "Honolulu"}}

State: California
{{"City": "Los Angeles"}}

State: {state}
"""

prompt = PromptTemplate.from_template(prompt_str)
chain = prompt | model


In [ ]:
response = chain.invoke({"state": "Georgia"})
response

In [ ]:
response.content

In [ ]:
import json
data = json.loads(response.content)
data

In [ ]:
data["City"]

In [ ]:
prompt_prefix = """What is the most populated city in the state provided below. 
Provide city name and no additional information. """


In [ ]:
prompt_examples = [
    {"ExampleState": "Hawaii", "ExampleCity": "Honolu"},
    {"ExampleState": "California", "ExampleCity": "Los Angeles"}   
]
prompt_examples

In [ ]:
example_prompt_str ="State: {ExampleState}\nCity: {ExampleCity}"
print(example_prompt_str)

In [ ]:
example_prompt = PromptTemplate(input_variables=["ExampleState", "ExampleCity"], template = example_prompt_str)
example_prompt


In [ ]:
print(example_prompt.format(**prompt_examples[0]))

In [ ]:
print(example_prompt.format(**prompt_examples[1]))

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

execute_fewshot_prompt = FewShotPromptTemplate(
    prefix = prompt_prefix,
    input_variables=["state"],
    examples= prompt_examples,
    example_prompt = example_prompt,
    example_separator="\n\n",
    suffix = "State: {state}"
)

In [ ]:
data = {"state": "Georgia"}
print(execute_fewshot_prompt.format(**data))

In [ ]:
chain = execute_fewshot_prompt | model
chain.invoke(data)

In [ ]:
example_prompt_str_json = """ State: {ExampleState}\n  {open_curly} "City": "{ExampleCity}" {close_curly} """
print(example_prompt_str_json)

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["ExampleState", "ExampleCity"],  
    partial_variables={"open_curly": "{{", "close_curly": "}}"},
    template = example_prompt_str_json)
example_prompt


In [ ]:
prompt_examples[0]

In [ ]:
print(example_prompt.format(**prompt_examples[1]))

In [ ]:
example_prompt

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

execute_fewshot_prompt = FewShotPromptTemplate(
    prefix = prompt_prefix,
    input_variables=["state"], 

    examples= prompt_examples,
    example_prompt = example_prompt,
    example_separator="\n\n",
    suffix = "State: {state}"
)



In [ ]:
data = {"state": "Georgia"}
print(execute_fewshot_prompt.format(**data))

In [ ]:
chain = execute_fewshot_prompt | model
response = chain.invoke(data)
response

In [ ]:
response.content

In [ ]:
data = json.loads(response.content)
data

In [ ]:
data['City']

In [ ]:
from pydantic import BaseModel, Field


In [ ]:
class CityParser(BaseModel):
    City: str = Field(..., description="The name of the most populous city") 

In [ ]:
from langchain.output_parsers import PydanticOutputParser
cityParser = PydanticOutputParser(pydantic_object=CityParser)


In [ ]:
cityParser.parse("""{"City": "Atlanta"}""")



In [ ]:
output = cityParser.parse("""{"City": "Atlanta"}""")
output.City


In [ ]:
data = {"state": "Georgia"}
chain = execute_fewshot_prompt | model | cityParser
reponse = chain.invoke(data)
reponse

In [ ]:
reponse.City


In [ ]:
data = {"state": "Georgia"}
print(execute_fewshot_prompt.format(**data))

In [ ]:
print(prompt_prefix)

In [ ]:
prompt_prefix = """What is the most populated city in the state provided below. 
Provide city name and no additional information. 
{format_instructions}
"""

In [ ]:
execute_fewshot_prompt = FewShotPromptTemplate(
    prefix = prompt_prefix,
    input_variables=["state"], 
    partial_variables={"format_instructions": cityParser.get_format_instructions()},
    examples= prompt_examples,
    example_prompt = example_prompt,
    example_separator="\n\n",
    suffix = "State: {state}\n"
)
data = {"state": "Georgia"}
print(execute_fewshot_prompt.format(**data))

In [ ]:
! pip install huggingface_hub

In [ ]:
# from getpass import getpass

# HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
from langchain.llms import HuggingFaceHub
repo_id_flan = "google/flan-t5-xxl" 


llm_google_flan = HuggingFaceHub(
    repo_id= repo_id_flan, model_kwargs={"temperature": 1, "max_length": 64},
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN
)

In [ ]:
data

In [ ]:
print(execute_fewshot_prompt.format(**data))

In [ ]:
chain = execute_fewshot_prompt | llm_google_flan 
reponse = chain.invoke(data)


In [ ]:
reponse

In [ ]:
from langchain.llms import HuggingFaceHub
# repo_id_Llama_2 = "meta-llama/Llama-2-13b-chat-hf"
repo_id_mistral = "mistralai/Mistral-7B-Instruct-v0.1" 


llm_google_mistral = HuggingFaceHub(
    repo_id= repo_id_mistral, model_kwargs={"temperature": 0.1, "max_length": 64},
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN
)

chain = execute_fewshot_prompt | llm_google_mistral 

reponse = chain.invoke(data)

reponse

In [ ]:
print(execute_fewshot_prompt.format(**data))

In [ ]:
chain = execute_fewshot_prompt | llm_google_mistral.bind(stop="\n")

reponse = chain.invoke(data)

reponse

In [ ]:
chain = execute_fewshot_prompt | llm_google_mistral.bind(stop="\n") | cityParser

reponse = chain.invoke(data)

reponse

- thoughts: Can you do receipts with one or do you need multiple prompts?
    
